# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder)

In [1]:
from dvats.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader

## Config parameters

Put here everything that could be needed if this notebook

In [2]:
config = AttrDict(
    use_wandb = False, # Whether to use or not wandb for experiment tracking
    wandb_group = 'embeddings', # Whether to group this run in a wandb group
    wandb_entity = os.environ['WANDB_ENTITY'], # The entity to use for wandb
    wandb_project = os.environ['WANDB_PROJECT'], # The project to use for wandb
    enc_artifact = 'deepvats/deepvats/mvp:v1', # Name:version of the encoder artifact
    input_ar = None, # If none, the validation set used to train enc_artifact is used
    cpu = False
)

## Run

In [3]:
run = wandb.init(entity=config.wandb_entity,
                    project=config.wandb_project if config.use_wandb else 'work-nbs', 
                    group=config.wandb_group,
                    job_type='embeddings', 
                    mode='online' if config.use_wandb else 'disabled',
                    anonymous = 'never' if config.use_wandb else 'must',
                    config=config,
                    #id = 'embeddingsProvider',
                    resume='allow')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [4]:
# Botch to use artifacts offline
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

Restore the encoder model and its associated configuration

In [5]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')

In [6]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()
enc_learner

Restore the dataset artifact used for training the encoder. Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that 
it matches with the dataset that we want to reduce.

In [9]:
enc_run = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_run.config['train_artifact'], type='dataset')
enc_artifact_train.name

'ABP:v0'

Now we specify the dataset artifact that we want to get the embeddings from. If no 
artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [10]:
input_ar_name = ifnone(config.input_ar, 
                       f'{enc_artifact_train.entity}/{enc_artifact_train.project}/{enc_artifact_train.name}')
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

'ABP:v0'

In [11]:
df = input_ar.to_df()
df

,abp
1970-01-01 00:00:00,6832.0
1970-01-01 00:00:01,6928.0
1970-01-01 00:00:02,6968.0
1970-01-01 00:00:03,6992.0
1970-01-01 00:00:04,6980.0
...,...
1970-01-01 11:06:35,3468.0
1970-01-01 11:06:36,3460.0
1970-01-01 11:06:37,3456.0
1970-01-01 11:06:38,3472.0


In [12]:
df.shape

(40000, 1)

In [13]:
enc_input, _ = SlidingWindow(window_len=enc_run.config['w'], 
                             stride=enc_run.config['stride'], 
                             get_y=[])(df)
enc_input.shape

(39791, 1, 210)

In [14]:
embs = get_enc_embs(enc_input, enc_learner, cpu=config.cpu, to_numpy=True)

CPU times: user 13.6 s, sys: 2.07 s, total: 15.7 s
Wall time: 19.4 s


In [15]:
if config.use_wandb: 
    run.log_artifact(ReferenceArtifact(embs, 'embeddings', metadata=dict(run.config)), 
                     aliases=f'run-{run.project}-{run.id}')

In [16]:
run.finish()